In [1]:
import requests
from bs4 import BeautifulSoup
url = "https://www.opindia.com/latest-news/"
data = requests.get(url)

In [2]:
data

<Response [200]>

In [3]:
soup = BeautifulSoup(data.content,"html.parser")

In [4]:
def fetch_article(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.content,"html.parser")
    articles = []
    for i in soup.find_all("h3",class_ = ["entry-title td-module-title"]):
        articles.append(i.find('a')['title'])     
    return articles

In [5]:
urllist = []
for i in range(2,21,1):
    url = "https://www.opindia.com/latest-news/page/" + str(i) + "/"
    urllist.append(url)

In [6]:
all_articles = []
for i in urllist:
    all_articles.extend(fetch_article(i))

In [19]:
all_articles

['IMF, which predicted India cannot become a $5 trillion economy before 2028-2029, changes its forecast: Reports',
 'Scratch a ‘vivadit dhancha’, find a Mandir: A list of Hindu temples destroyed over centuries of Islamic rule where masjids and dargahs stand now',
 'Two different Ideas of India: ‘Bharat Mata Ki Jai’ chants in Bangkok and Rahul Gandhi’s argument that India doesn’t exist as a ‘nation’',
 'AIMIM leader Asaduddin Owaisi provokes Muslims against court-ordered proceedings after Shivling discovered inside the disputed Gyanvapi structure',
 'Srinagar Mayor hails Aurangzeb, prays for mercy and blessings after Devendra Fadnavis’ ‘kutta bhi nahi peshab karega’ speech for Mughal tyrant’s grave',
 'Ghazwa-e-Hind: The Islamic prophecy that explains why the likes of Owaisi feel proud of Mughal tyrant Aurangzeb and his anti-Hindu deeds',
 'Rahul Bhat was anything but a ‘Sanghi’: How his murder shows the ‘political’ problem of Kashmir is nothing but a cover for Jihad',
 'Two different I

In [7]:
p_art =[]
for i in all_articles:
    q = i.upper()
    import re
    q = re.sub("[^A-Z0-9 ]","",q)
    from nltk.stem import PorterStemmer
    tk_q = q.split(" ")
    sent = ""
    for j in tk_q:
        ps = PorterStemmer()
        sent = sent + " " + ps.stem(j).upper()
    p_art.append(sent)

In [20]:
p_art

[' IMF WHICH PREDICT INDIA CANNOT BECOM A 5 TRILLION ECONOMI BEFOR 20282029 CHANG IT FORECAST REPORT',
 ' SCRATCH A VIVADIT DHANCHA FIND A MANDIR A LIST OF HINDU TEMPL DESTROY OVER CENTURI OF ISLAM RULE WHERE MASJID AND DARGAH STAND NOW',
 ' TWO DIFFER IDEA OF INDIA BHARAT MATA KI JAI CHANT IN BANGKOK AND RAHUL GANDHI ARGUMENT THAT INDIA DOESNT EXIST AS A NATION',
 ' AIMIM LEADER ASADUDDIN OWAISI PROVOK MUSLIM AGAINST COURTORD PROCEED AFTER SHIVL DISCOV INSID THE DISPUT GYANVAPI STRUCTUR',
 ' SRINAGAR MAYOR HAIL AURANGZEB PRAY FOR MERCI AND BLESS AFTER DEVENDRA FADNAVI KUTTA BHI NAHI PESHAB KAREGA SPEECH FOR MUGHAL TYRANT GRAVE',
 ' GHAZWAEHIND THE ISLAM PROPHECI THAT EXPLAIN WHI THE LIKE OF OWAISI FEEL PROUD OF MUGHAL TYRANT AURANGZEB AND HI ANTIHINDU DEED',
 ' RAHUL BHAT WA ANYTH BUT A SANGHI HOW HI MURDER SHOW THE POLIT PROBLEM OF KASHMIR IS NOTH BUT A COVER FOR JIHAD',
 ' TWO DIFFER IDEA OF INDIA BHARAT MATA KI JAI CHANT IN BANGKOK AND RAHUL GANDHI ARGUMENT THAT INDIA DOESNT EXIST 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
A = tf.fit_transform(p_art).toarray()

In [21]:
A

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.29583243,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [48]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5)
cl_res = km.fit(A)

In [49]:
cl_res.labels_

array([0, 0, 4, ..., 1, 1, 1])

In [50]:
import pandas as pd
Q = pd.DataFrame(p_art,columns=["Article"])
Q['Cluster']=cl_res.labels_

In [13]:
E = {1 : "politics",
2 : "religion",
3 : "sports",
4 : "entertainment",
0 : "geopolitics"}

In [14]:
R = []
for i in Q.Cluster:
    R.append(E[i])

Q['category'] = R

In [15]:
Q

,Article,Cluster,category
0,IMF WHICH PREDICT INDIA CANNOT BECOM A 5 TRIL...,0,geopolitics
1,SCRATCH A VIVADIT DHANCHA FIND A MANDIR A LIS...,3,sports
2,TWO DIFFER IDEA OF INDIA BHARAT MATA KI JAI C...,0,geopolitics
3,AIMIM LEADER ASADUDDIN OWAISI PROVOK MUSLIM A...,1,politics
4,SRINAGAR MAYOR HAIL AURANGZEB PRAY FOR MERCI ...,4,entertainment
...,...,...,...
1971,RAVISH KUMAR UNHING AGAIN USE THE ALIA BHATTR...,2,religion
1972,INDIA TO GET A RECORD NUMBER OF NEW DOCTOR IN...,2,religion
1973,UP NAEEM AND MUSTKIM PLAY PAKISTAN ZINDABAD S...,2,religion
1974,JHARKHAND LOHARDAGA RIOT PREPLAN THERE A HUGE...,2,religion
